# Spark SQL Examples

Run the code cells below. This is the same code from the previous screencast.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import desc
from pyspark.sql.functions import asc
from pyspark.sql.functions import sum as Fsum

import datetime

import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

Define a new Spark Session

In [4]:
spark = SparkSession \
    .builder \
    .appName("Data wrangling with Spark SQL") \
    .getOrCreate()

Load the same old Titanic data (in json instead of csv)

In [5]:
path = "titanic.json"
df = spark.read.json(path)

Check couple of rows

In [6]:
df.take(2)

[Row(Age=22.0, Cabin=None, Embarked='S', Fare=7.25, Name='Braund, Mr. Owen Harris', Parch=0, PassengerId=1, Pclass=3, Sex='male', SibSp=1, Survived=0, Ticket='A/5 21171'),
 Row(Age=38.0, Cabin='C85', Embarked='C', Fare=71.2833, Name='Cumings, Mrs. John Bradley (Florence Briggs Thayer)', Parch=0, PassengerId=2, Pclass=1, Sex='female', SibSp=1, Survived=1, Ticket='PC 17599')]

If you are not aware about the schema then check it through printschema

In [7]:
df.printSchema()

root
 |-- Age: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Name: string (nullable = true)
 |-- Parch: long (nullable = true)
 |-- PassengerId: long (nullable = true)
 |-- Pclass: long (nullable = true)
 |-- Sex: string (nullable = true)
 |-- SibSp: long (nullable = true)
 |-- Survived: long (nullable = true)
 |-- Ticket: string (nullable = true)



# Create a View And Run Queries

The code below creates a temporary view against which we can run SQL queries.

In [8]:
df.createOrReplaceTempView("titanic_table")

Check data through SQL query

In [10]:
spark.sql("SELECT * FROM titanic_table LIMIT 2").show()

+----+-----+--------+-------+--------------------+-----+-----------+------+------+-----+--------+---------+
| Age|Cabin|Embarked|   Fare|                Name|Parch|PassengerId|Pclass|   Sex|SibSp|Survived|   Ticket|
+----+-----+--------+-------+--------------------+-----+-----------+------+------+-----+--------+---------+
|22.0| null|       S|   7.25|Braund, Mr. Owen ...|    0|          1|     3|  male|    1|       0|A/5 21171|
|38.0|  C85|       C|71.2833|Cumings, Mrs. Joh...|    0|          2|     1|female|    1|       1| PC 17599|
+----+-----+--------+-------+--------------------+-----+-----------+------+------+-----+--------+---------+



we can use above format or below format to run SQL queries

In [11]:
spark.sql('''
          SELECT * 
          FROM titanic_table 
          LIMIT 2
          '''
           ).show()

+----+-----+--------+-------+--------------------+-----+-----------+------+------+-----+--------+---------+
| Age|Cabin|Embarked|   Fare|                Name|Parch|PassengerId|Pclass|   Sex|SibSp|Survived|   Ticket|
+----+-----+--------+-------+--------------------+-----+-----------+------+------+-----+--------+---------+
|22.0| null|       S|   7.25|Braund, Mr. Owen ...|    0|          1|     3|  male|    1|       0|A/5 21171|
|38.0|  C85|       C|71.2833|Cumings, Mrs. Joh...|    0|          2|     1|female|    1|       1| PC 17599|
+----+-----+--------+-------+--------------------+-----+-----------+------+------+-----+--------+---------+



Check total rows through SQL count function 

In [12]:
spark.sql('''
          SELECT COUNT(*) 
          FROM titanic_table 
          '''
          ).show()

+--------+
|count(1)|
+--------+
|     891|
+--------+



Check other SQL functions

In [13]:
spark.sql('''
          SELECT Name, Age, Survived
          FROM titanic_table 
          WHERE Sex == 'male'
          '''
          ).collect()

[Row(Name='Braund, Mr. Owen Harris', Age=22.0, Survived=0),
 Row(Name='Allen, Mr. William Henry', Age=35.0, Survived=0),
 Row(Name='Moran, Mr. James', Age=None, Survived=0),
 Row(Name='McCarthy, Mr. Timothy J', Age=54.0, Survived=0),
 Row(Name='Palsson, Master. Gosta Leonard', Age=2.0, Survived=0),
 Row(Name='Saundercock, Mr. William Henry', Age=20.0, Survived=0),
 Row(Name='Andersson, Mr. Anders Johan', Age=39.0, Survived=0),
 Row(Name='Rice, Master. Eugene', Age=2.0, Survived=0),
 Row(Name='Williams, Mr. Charles Eugene', Age=None, Survived=1),
 Row(Name='Fynney, Mr. Joseph J', Age=35.0, Survived=0),
 Row(Name='Beesley, Mr. Lawrence', Age=34.0, Survived=1),
 Row(Name='Sloper, Mr. William Thompson', Age=28.0, Survived=1),
 Row(Name='Emir, Mr. Farred Chehab', Age=None, Survived=0),
 Row(Name='Fortune, Mr. Charles Alexander', Age=19.0, Survived=0),
 Row(Name='Todoroff, Mr. Lalio', Age=None, Survived=0),
 Row(Name='Uruchurtu, Don. Manuel E', Age=40.0, Survived=0),
 Row(Name='Wheadon, Mr. 

In [14]:
spark.sql('''
          SELECT DISTINCT Cabin
          FROM titanic_table 
          ORDER BY Cabin ASC
          '''
          ).show()

+-----+
|Cabin|
+-----+
| null|
|  A10|
|  A14|
|  A16|
|  A19|
|  A20|
|  A23|
|  A24|
|  A26|
|  A31|
|  A32|
|  A34|
|  A36|
|   A5|
|   A6|
|   A7|
| B101|
| B102|
|  B18|
|  B19|
+-----+
only showing top 20 rows



# User Defined Functions

We first need to register the udf function before we can use it in SQL

In [15]:
spark.udf.register("name_prefix", lambda x: x.split(',')[1].strip(" ").split(" ")[0].strip(" "))

<function __main__.<lambda>(x)>

In [16]:
spark.sql('''
          SELECT *, name_prefix(Name) AS Name_Prfx
          FROM titanic_table 
          LIMIT 5
          '''
          ).collect()

[Row(Age=22.0, Cabin=None, Embarked='S', Fare=7.25, Name='Braund, Mr. Owen Harris', Parch=0, PassengerId=1, Pclass=3, Sex='male', SibSp=1, Survived=0, Ticket='A/5 21171', Name_Prfx='Mr.'),
 Row(Age=38.0, Cabin='C85', Embarked='C', Fare=71.2833, Name='Cumings, Mrs. John Bradley (Florence Briggs Thayer)', Parch=0, PassengerId=2, Pclass=1, Sex='female', SibSp=1, Survived=1, Ticket='PC 17599', Name_Prfx='Mrs.'),
 Row(Age=26.0, Cabin=None, Embarked='S', Fare=7.925, Name='Heikkinen, Miss. Laina', Parch=0, PassengerId=3, Pclass=3, Sex='female', SibSp=0, Survived=1, Ticket='STON/O2. 3101282', Name_Prfx='Miss.'),
 Row(Age=35.0, Cabin='C123', Embarked='S', Fare=53.1, Name='Futrelle, Mrs. Jacques Heath (Lily May Peel)', Parch=0, PassengerId=4, Pclass=1, Sex='female', SibSp=1, Survived=1, Ticket='113803', Name_Prfx='Mrs.'),
 Row(Age=35.0, Cabin=None, Embarked='S', Fare=8.05, Name='Allen, Mr. William Henry', Parch=0, PassengerId=5, Pclass=3, Sex='male', SibSp=0, Survived=0, Ticket='373450', Name_Pr

We can see Name_Prfx column in the above result

In [17]:
prefix_count = spark.sql('''
          SELECT name_prefix(Name) AS Name_Prfx, COUNT(*) as count_by_prefix
          FROM titanic_table
          WHERE Survived = 1
          GROUP BY Name_Prfx
          ORDER BY count_by_prefix DESC
          '''
          )

In [18]:
prefix_count.show()

+---------+---------------+
|Name_Prfx|count_by_prefix|
+---------+---------------+
|    Miss.|            127|
|     Mrs.|             99|
|      Mr.|             81|
|  Master.|             23|
|      Dr.|              3|
|    Mlle.|              2|
|      Ms.|              1|
|   Major.|              1|
|     Sir.|              1|
|     Col.|              1|
|     Mme.|              1|
|      the|              1|
|    Lady.|              1|
+---------+---------------+



# Converting Results to Pandas

We can easily covert the results to Pandas

In [20]:
prefix_count_pd = prefix_count.toPandas()

In [21]:
print(prefix_count_pd)

   Name_Prfx  count_by_prefix
0      Miss.              127
1       Mrs.               99
2        Mr.               81
3    Master.               23
4        Dr.                3
5      Mlle.                2
6        Ms.                1
7       Sir.                1
8     Major.                1
9       Col.                1
10      Mme.                1
11       the                1
12     Lady.                1
